In [1]:
import pandas as pd
import numpy as np
import json, csv, datetime

In [14]:
#input

file_path = '/home/adrian/Dokumente/real_data/IDAWEB/idaweb3/order47741/data3.csv'

#output
output_path = '/home/adrian/Dokumente/real_data/idaweb_out_s3/'

test_file_name = output_path + 'test.csv'
train_file_name = output_path + 'train.csv'

In [15]:
def load_data_frame(file_path):
    df = pd.read_csv(file_path,
                     sep=';',
                     parse_dates=['time'],
                     infer_datetime_format=True,
                     index_col='time',
                     float_precision='high')
    return df

In [16]:
def init_file(outputfile):
    with open(outputfile, 'w') as file_output:
        writer = csv.writer(file_output, delimiter=';')
        header = ['ts_name', 'time', 'unit', 'value', 'class']
        writer.writerow(header)
        
def append_to_file(df, ts_name, unit, outputfile):
    with open(outputfile, 'a') as file_output:
        writer = csv.writer(file_output, delimiter=';')
        for index, value in df.iteritems():
            row = [ts_name, index.isoformat(), unit, value, 0]
            writer.writerow(row)

In [17]:
ts_names_9 = ['LUZ', 'RGNOT', 'MOA', 'KOP', 'LAG', 'OBR', 'LAE', 'ORO', 'PAY']

In [5]:
df_read = load_data_frame(file_path)

In [9]:
df = df_read.pivot(columns='ts_name', values='value')

In [18]:
def generate_output(df, ts_names, path_addition, file_suffix, ts_length=1000, training_length=500):
    test_file_name = output_path + path_addition + 'test_' + file_suffix + '.csv'
    train_file_name = output_path + path_addition + 'train_' + file_suffix + '.csv'
    init_file(test_file_name)
    init_file(train_file_name)
    
    for ts_name in ts_names:
        df_set = df[ts_name]
        df_test = df_set.iloc[0:1500]
        df_train = df_set.iloc[1500:3000]
    
        append_to_file(df_test, ts_name, 'g/kg', test_file_name)
        append_to_file(df_train, ts_name, 'g/kg', train_file_name)

In [19]:
generate_output(df, ts_names_9, 'set3/', file_suffix='9')

In [7]:
#Correlation

df = df_read.pivot(columns='ts_name', values='value')
df = df[0:8000]
df_no_na = df.dropna(axis=1, how='any')
print(df_no_na.head())
df_corr = df_no_na.corr(method='pearson')
df_corr = df_corr.round(2)
print(df_corr.head())
df_corr.to_csv(output_path + 'set3/' +'corr.csv', index = True, header=True)

ts_name              ATT  BEH  BRL  CDF  CGI  CHB  COV  DOL  GEN  INT  ...  \
time                                                                   ...   
2016-06-01 00:00:00  4.8  5.3  4.8  5.8  7.4  6.9  4.5  5.7  7.1  9.1  ...   
2016-06-01 00:10:00  4.9  5.4  4.8  5.4  7.3  6.8  4.4  5.7  7.1  9.1  ...   
2016-06-01 00:20:00  4.9  5.3  4.8  5.5  7.3  6.7  4.5  5.7  7.1  9.1  ...   
2016-06-01 00:30:00  4.9  5.4  4.6  5.7  7.2  6.4  4.5  5.7  7.1  9.1  ...   
2016-06-01 00:40:00  5.0  5.4  4.5  5.9  7.2  6.4  4.5  5.7  7.1  9.2  ...   

ts_name              NAP  NAS  NEU  OBR  ORO  OTL  PAY  PIL  PIO  RGNOT  
time                                                                     
2016-06-01 00:00:00  6.5  5.4  7.9  9.8  7.1  9.0  7.2  5.8  8.1    8.5  
2016-06-01 00:10:00  6.6  5.6  7.9  9.7  7.0  8.9  7.4  5.8  8.1    8.5  
2016-06-01 00:20:00  6.5  5.5  7.8  9.6  7.0  8.9  7.2  5.9  8.0    8.4  
2016-06-01 00:30:00  6.4  5.5  7.8  9.7  7.0  8.9  7.5  5.9  8.0    8.3  
2016-06-0

In [9]:
print(df)

ts_name             WSLBAF WSLBTF WSLCIF WSLCLF WSLISF WSLJUF WSLLAF WSLNAF  \
time                                                                          
2017-04-01 00:00:00      0   -0.1    NaN   -0.3   2659    4.2      2    0.4   
2017-04-01 00:10:00      0      0    NaN   -2.6   2657    4.2      1    0.4   
2017-04-01 00:20:00      0      0    NaN   -0.1   2695    4.3    0.6    0.4   
2017-04-01 00:30:00      0      0    NaN   -1.1      -    4.2      1    0.4   
2017-04-01 00:40:00      0      0    NaN   -0.3      -    4.2    4.1    0.4   
...                    ...    ...    ...    ...    ...    ...    ...    ...   
2017-04-30 23:10:00      0      0      -   -0.1   2428    4.4    1.9    0.8   
2017-04-30 23:20:00      0      0      -   -1.4   2366    4.3    0.4    0.9   
2017-04-30 23:30:00      0      0      -   -0.7   2357    4.3    3.1      1   
2017-04-30 23:40:00      0      0      -   -1.1   2350    4.3    1.4    0.7   
2017-04-30 23:50:00      0      0      -   -0.5   23

In [7]:
def create_set(path, files, file_set_names, file_suffix):
    test_file_name = output_path + path + 'test_' + file_suffix + '.csv'
    train_file_name = output_path + path + 'train_' + file_suffix + '.csv'
    init_file(test_file_name)
    init_file(train_file_name)
    
    for entry in files:
        ts_name = entry['name']
        if ts_name in file_set_names:
            file_path = entry['file_path']
            df = load_data_frame(ts_name, file_path)
            df = time_index_to_dt(df)
            print(ts_name + "\t anomalies " 
                  + str(df.loc[df['class']==1].shape[0]) 
                  + '\t values ' + str(df.shape[0])
                  + '\t min ' + str(df.loc[df['class']==0, 'value'].min()) 
                  + '\t max ' + str(df.loc[df['class']==0, 'value'].max()))
            df_test = df.iloc[0:710]
            print('test' + "\t anomalies " + str(df_test.loc[df_test['class']==1].shape[0]) + '\t values ' + str(df_test.shape[0]))

            df_train = df.iloc[710:1421]
            print('train' + "\t anomalies " + str(df_train.loc[df_train['class']==1].shape[0]) + '\t values ' + str(df_train.shape[0]))
            print('\n')
            append_to_file(df_test, test_file_name)
            append_to_file(df_train, train_file_name)